In [113]:
from logging import info
from itertools import chain

In [143]:
def chain_list(_list: list):
    return list(chain.from_iterable(_list))

In [144]:
examples=[["a + a = b", "b - d = c", "a + b = d"],
          ["a + 3g = k", "-70a = g"],
          ["-j -j -j + j = b"]
         ]
formula=["c + a + b",
         "-k + a",
         "-j - b"
        ]
answer=["2a",
        "210a",
        "1j"
       ]

for i in range(len(answer)):
    print("example", i+1)
    print("\t", 'equations:', str(examples[i]))
    print("\t", 'formula:', str(formula[i]))
    print("\t", 'expected answer:', str(answer[i]))
exp_chain = [x.split("=")[0] for x in chain_list(examples)]
all_exp = formula + exp_chain
all_exp

example 1
	 equations: ['a + a = b', 'b - d = c', 'a + b = d']
	 formula: c + a + b
	 expected answer: 2a
example 2
	 equations: ['a + 3g = k', '-70a = g']
	 formula: -k + a
	 expected answer: 210a
example 3
	 equations: ['-j -j -j + j = b']
	 formula: -j - b
	 expected answer: 1j


['c + a + b',
 '-k + a',
 '-j - b',
 'a + a ',
 'b - d ',
 'a + b ',
 'a + 3g ',
 '-70a ',
 '-j -j -j + j ']

In [145]:

def rm_spaces(input: str):
    return "".join(input.split(" "))

def cum_prod(int_list):
    return_val = 1
    for val in int_list:
        return_val *= int(val)
    return return_val

def cum_prod_prod_str(int_list, cur_val):
    try:
        cur_val = int(cur_val)
    except ValueError:
        if cur_val == "":
            cur_val = 1
        elif cur_val == "-":
            cur_val = -1
    return str(cum_prod(int_list) * cur_val)

In [146]:
class Term:
    def __init__(self, val: int = 1, var: str = "", raw: str = None):
        self.val = val
        self.var = var
        if raw is not None:
            raw = raw.strip(" ")
            if raw[-1].isalpha():
                self.var = raw[-1]               
            else:
                self.var = "int"         
            if self.var == "int":
                try:
                    self.val = int(raw)
                except ValueError:
                    if raw == "-":
                        self.val = -1
                    info(raw)
            elif (len(raw) == 1 and raw.isalpha()):
                self.val = 1
            elif (len(raw) == 2 and raw[0] == "-"):
                self.val = -1
            else:
                self.val = int(raw[:-1])

    def __repr__(self):
        if self.var == "int":
            return str(self.val)
        return str(self.val) + self.var

    def __add__(self, term):
        if self.var == term.var:
            new_val = int(self.val) + int(term.val)
            return Term(var=self.var, val=new_val)
        else:
            raise ValueError(f"the vars don't match: {self.var} != {term.var}")
    
    def __mul__(self, integer):
        self.val *= integer
    
    def __neg__(self):
        self.val = -self.val

-1a a -1
123y y 123
1b b 1
34f f 34
-234 int -234


In [197]:
class Expression:
    def _get_vars(self):
        letters = [x for x in self.exp if x.isalpha()]
        return list(set(letters))

    def _rm_parens(self, exp):
        plus_exp = ""
        for i, char in enumerate(exp):
            if (char == "-" and i > 0):
                plus_exp =  plus_exp + "+"
            elif (i > 0 and char == "(" and exp[i - 1] == "-"):
                plus_exp =  plus_exp + "1"
            elif (char.isalpha() and (exp[i - 1] in "+-(" or i == 0)):
                plus_exp =  plus_exp + "1"
            plus_exp =  plus_exp + char
            info("add_plus_str:", plus_exp)
        
        int_list = [1]
        info("plus_exp", plus_exp)
        plus_list = plus_exp.split("+")
        new_list = []
        for i, term in enumerate(plus_list):
            if "(" in term:
                term = term.split("(")
                if term[0] in ["-", "-1"]:
                    to_append = -1
                elif term[0] in ["1", ""]:
                    to_append = 1
                else:
                    to_append = int(term[0])
                int_list.append(to_append)
                if term[-1] == "":
                    continue
                else:
                    term = term[-1]
            info(term, new_list, int_list)
            if ")" in term:
                right_paren_ind = True
                term = term.strip(")")
            else:
                right_paren_ind = False            
            if term in ["", "+", "-"]:
                continue   
            if term[-1].isalpha():
                var = term[-1]
                if len(term) == 1:
                    _val = 1
                elif (len(term) == 2 and term[0] == "-"):
                    _val = -1
                else:
                    _val = term[:-1]
            else:
                var = ""
                _val = int(term)
                
            val = cum_prod_prod_str(int_list, _val)
            info(f"log{i}: term={plus_list[i]} var={var}, val={val}")
            if right_paren_ind:
                int_list.pop(-1)
            new_list.append(val + var)
        return new_list

    def make_term_dict(self):
        term_dict = {}
        for i, term in enumerate(self.raw_terms):
            if term == "":
                continue
            _term = Term(raw=term)
            cur_keys = list(term_dict.keys())
            if _term.var in cur_keys:
                term_dict[_term.var] = term_dict[_term.var] + _term         
            else:
                term_dict[_term.var] = _term
        return term_dict

    def __init__(self, expression: str):
        self.exp = rm_spaces(expression)
        info("post rm_spaces", self.exp)
        self.raw_terms = self._rm_parens(self.exp)
        info("post rm_parens", self.raw_terms)
        self.term_dict = self.make_term_dict()
        info("term_dict:", self.term_dict)
        self.vars = self._get_vars()

    def __repr__(self):
        term_objs = list(self.term_dict.values())
        term_repr = [str(x) for x in term_objs if x.val != 0]
        info("term_reprs:", term_repr)
        return "+".join(term_repr)
     
#for exp in all_exp[5:]:
for exp in ["-3(-70a)", "-(1a + 3a -1a +(4a -5a) + 6a)"]:
    print("original expression:", exp)
    e = Expression(exp)
    print("object repr:", e)

original expression: -3(-70a)
object repr: 210a
original expression: -(1a + 3a -1a +(4a -5a) + 6a)
object repr: -8a


In [198]:
class PreSimplifier:
    def get_eq_dict(self):
        eqs = [eq.split("=") for eq in self.equations]
        _dict = {}
        for eq in eqs:
            if len(eq[0]) == 1:
                key = eq[0]
                val = eq[-1]
            elif len(eq[-1]) == 1:
                key = eq[-1]
                val = eq[0]
            else:
                raise ValueError("equation not one-sided")
            _dict[key] = f"{Expression(val)}"
        return _dict
    
    def __init__(self, equations, formula):
        self.equations = [rm_spaces(x) for x in equations]
        self.formula = Expression(rm_spaces(formula))
        self.eq_dict = self.get_eq_dict()
        for eq in list(self.eq_dict.keys()):
            self.eq_dict[eq] = Expression(self.eq_dict[eq])

In [199]:
def sub_var_into_str_exp(str_exp: str, sub_var: str, insert_str: str):
    split_exp = str_exp.split(sub_var)
    return insert_str.join(split_exp)

class Simplifier: 
    def __init__(self, equations, formula):
        ps = PreSimplifier(equations, formula)
        self.eq_dict = ps.eq_dict
        self.formula = ps.formula
        self.eq_keys = list(self.eq_dict.keys())
    
    def one_formula_sub_loop(self):
        for var in self.formula.vars:
            if var in self.eq_keys:
                str_exp = str(self.formula)
                insert_str = f"({self.eq_dict[var]})"
                new_exp = sub_var_into_str_exp(str_exp, var, insert_str)
                self.formula = Expression(new_exp)
                info("new exp:", new_exp, "new form:", self.formula)

In [200]:

def simplify(eqs: list, form: str):
    simp = Simplifier(eqs, form)
    prev_formula = ""
    cur_formula = str(simp.formula)
    loop_counter = 0
    while (prev_formula != cur_formula and loop_counter < 3):
        simp.one_formula_sub_loop()
        prev_formula = cur_formula
        cur_formula = str(simp.formula)
        loop_counter += 1
    return cur_formula
for i in range(3):
    simp = simplify(examples[i], formula[i])
    print("real answer:", answer[i])
    print("calc answer:", simp)


real answer: 2a
calc answer: 2a
real answer: 210a
calc answer: 210a
real answer: 1j
calc answer: 1j


In [159]:
i = ex_num
print("example", i + 1)
print("\t", 'equations:', str(examples[i]))
print("\t", 'formula:', str(formula[i]))
print("\t", 'expected answer:', str(answer[i]))

example 2
	 equations: ['a + 3g = k', '-70a = g']
	 formula: -k + a
	 expected answer: 210a
